**Trabajo MOVIX G16**  
*Inteligencia Artificial I*


Inteligencia Artificial I
Práctica 1.
(Sesión 3) Búsqueda heurística
El objetivo del puzle MovIX es utilizar el mínimo número de movimientos para
alinear (horizontal, vertical o diagonal) en un tablero NxN un conjunto X (X<= N) de
fichas con diferentes habilidades de movimiento.
Las celdas del tablero inicial pueden estar vacías o contener una ficha. El jugador
puede mover una ficha en cada turno y dos fichas no pueden ocupar la misma
casilla al mismo tiempo.


Hay varios tipos de fichas que tienen distintas habilidades de movimiento:

S- Saltadora: puede saltar sobre una ficha adyacente (horizontal o vertical)
y aterrizar en la siguiente celda vacía. No puede moverse si no es saltando
sobre una ficha, es decir, no se puede saltar sobre celdas vacías, ni sobre
dos fichas, ni sobre un muro (si se incluyen).


L- Lenta: puede moverse únicamente a una celda adyacente vacía en
cualquier dirección (horizontal, vertical o diagonal).


V- Vertical: puede moverse verticalmente a cualquier celda vacía en la
misma columna sin saltar fichas.


H- Horizontal: puede moverse horizontalmente a cualquier celda vacía en
la misma fila sin saltar fichas. Ejemplo: Si está en (1, 2) y toda la fila 1 está
vacía podría moverse a (1, 1), (1, 3), (1, 4), (1, 0).


La siguiente configuración es un ejemplo de la posición inicial de las fichas (al
inicio se coloca aleatoriamente un número dado (Y) de fichas de cada tipo. En el
ejemplo Y=1:
Se pide:


o Representa el problema en AIMA para resolverlo con búsqueda
heurística. Analiza el tamaño y forma del espacio de estados.


o Define varias H’ para el problema y analízalas comparando su
rendimiento al resolver distintos estados iniciales y distinto número
de fichas iniciales.

o Realiza pruebas con distintos tamaños de tablero N, de línea X, y
número de fichas iniciales de cada tipo (Y).


o Elige de forma justificada cual es la mejor h’ para este problema
o Compara la búsqueda heurística con algún algoritmo de búsqueda
ciega para medir las ventajas del uso de la heurística.


o Añade un nuevo tipo de casillas M muro sobre las que no se puede
saltar, es decir, un muro detiene el movimiento de las fichas V
vertical, H horizontal. Prueba el efecto que tiene en la resolución de
tableros usando las mismas heurísticas.


Ideas para definir heurísticas:


 Ten en cuenta si hay fichas ya alineadas y cuántas fichas adicionales se
necesitarían para completar la alineación de X fichas.


 Considera los conflictos entre fichas que impiden la alineación. Cada ficha
que está en la línea de alineación pero que no puede moverse (por ser de
un tipo que no permite el movimiento adecuado) suma un costo a la
heurística. Cuenta cuántas fichas están en la misma línea que no pueden
alinearse debido a sus restricciones de movimiento. Cada conflicto podría
sumar un valor (por ejemplo, 1 por cada conflicto).


 Combina las heurísticas anteriores para dar un valor más integral. Podrías
ponderar la distancia de Manhattan y la alineación parcial para crear una
heurística más completa. 

In [2]:

import numpy as np


from search import *

class MovIXNode(Node):
        def __new__(cls, input_array):
            # Create a new instance of the class without changing dtype
            obj = super().__new__(cls)  # Create a new instance of MovIXNode
            obj.state = np.asarray(input_array)  # Store the input array as a property
            return obj  # Return the new instance

        def __hash__(self):
            # Hash based on the bytes representation of the array
            return hash(self.state.tobytes())

        def __eq__(self, other):
            # Equality check
            if isinstance(other, MovIXNode):
                return np.array_equal(self, other)
            return False
        
        def __len__(self):  #devuelve la longitud del estado (número de piezas o filas), he hecho esto para poder acceder desde la heursitica
            return len(self.state)
        
        #comento el metodo iter, porque si no el kernel peta :(
        """def __iter__(self):  
            # Permite iterar sobre el estado (suponiendo que es un array 2D)
            return iter(self.state)"""





class MovIX(Problem):

    def __init__(self, initial, numpieces, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        dtype = [('piece', '<U1'), ('location', '<i4', 2)]
        initial_data = np.zeros((numpieces), dtype=dtype)
        self.InicializarEstadoInicial(initial, initial_data)
        self.initial = MovIXNode(initial_data)
        self.X = numpieces
        self.N = len(initial)
        self.goal = goal

    def actions(self, state):
        N = self.N  # Tamaño del tablero (NxN)
        possible_actions = []
        piezas = state.state['piece']  # Todas las piezas
        posiciones = state.state['location']  # Todas las posiciones de las piezas
        
        for i, pieza in enumerate(piezas):
            tipo = pieza  # El tipo de pieza (S, L, V, H)
            posicion = posiciones[i]  # La posición actual de la ficha (i, j)
            x, y = posicion  # Coordenadas de la ficha

            if tipo == 'S':  # Ficha Saltadora
                # Saltar sobre una ficha adyacente en horizontal o vertical
                adyacentes = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)] #Por orden, arriba, abajo, izquierda, derecha
                for adj_x, adj_y in adyacentes: #adj_x y adj_y son las coordenadas de las posiciones adyancentes x e y respectivamente
                    # Verificamos que haya una ficha adyacente y que la siguiente celda esté vacía
                    if 0 <= adj_x < N and 0 <= adj_y < N: #Compruebo que la casilla este dentro del tablero
                        # Calcular la posición a la que saltaría (debe estar vacía)
                        salto_x = adj_x + (adj_x - x) #mide la distancia entre las casillas y luego se suma otra vez para "saltarla"
                        salto_y = adj_y + (adj_y - y)
                        if (0 <= salto_x < N and 0 <= salto_y < N and 
                            self.is_empty(state, (salto_x, salto_y)) and
                            not self.is_empty(state, (adj_x, adj_y))):
                            possible_actions.append((i, (salto_x, salto_y)))

            elif tipo == 'L':  # Ficha Lenta
                # Puede moverse a una celda adyacente vacía en cualquier dirección
                movimientos = [(x-1, y), (x+1, y), (x, y-1), (x, y+1), (x-1, y-1), (x-1, y+1), (x+1, y-1), (x+1, y+1)] #incluye diagonales
                for mov_x, mov_y in movimientos:
                    if 0 <= mov_x < N and 0 <= mov_y < N and self.is_empty(state, (mov_x, mov_y)):
                        possible_actions.append((i, (mov_x, mov_y)))

            elif tipo == 'V':  # Ficha Vertical
                # Puede moverse en la misma columna hacia arriba o hacia abajo
                for mov_x in range(N):
                    if mov_x != x and self.is_empty(state, (mov_x, y)):
                        possible_actions.append((i, (mov_x, y)))

            elif tipo == 'H':  # Ficha Horizontal
                # Puede moverse en la misma fila hacia la izquierda o derecha
                for mov_y in range(N):
                    if mov_y != y and self.is_empty(state, (x, mov_y)):
                        possible_actions.append((i, (x, mov_y)))

        return possible_actions  # Aquí el return está dentro del método 'actions'

    def result(self, state, action):
        """Return the state that results from executing the given action in the given state."""
        # Extraer el índice de la pieza y la nueva posición de la acción
        index, new_position = action  # action es una tupla (índice de la pieza, nueva posición)
        
        # Creamos una copia del estado actual para no modificar el original
        new_state = state.state.copy()
        
        # Obtener la posición actual de la pieza
        current_position = state.state['location'][index]
        
        # Actualizar la nueva ubicación de la pieza
        new_state['location'][index] = new_position
        
        return MovIXNode(new_state)
        


    def goal_test(self, state)-> bool:
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""

        '''La función estudia si los puntos donde están colocados las piezas forman una semirrecta sin huecos.
        Para ello calcula las distancias de todos los puntos en las dos coordenadas, así como
        la pendiente de todos los puntos respecto al primero de los puntos.
        Si la pendiente es constante y dentro de los valores posibles según las restricciones del estado objetivo
        (m=+-1 para las diagonales, 0 para las horizontales o Inf para las verticales), los puntos están alineados.
        Además, si la distancia entre los valores no excede el número de puntos, se tiene una solución (ya que al no poder haber repetidos,
        X puntos alineados que no exceden la distancia X al primero por fuerza forman la semirrecta sin huecos)'''

        if self.X <= 1: return True #Si sólo hay un punto la semirrecta es trivial

        posiciones = state.state['location']#Tomamos solo las posiciones de las piezas

        firstPiece, secondPiece = posiciones[0], posiciones[1]#Calculamos la pendiente entre las dos primeras
       
        #En python no existe el do while :(

        firsti, firstj = firstPiece

        secondi, secondj = secondPiece

        # Si la recta es vertical la pendiente será infinita y no queremos que salte un warning
        np.seterr(divide='ignore', invalid='ignore')

        deltaj, deltai = (secondj - firstj), (secondi - firsti)

        currentSlope = np.divide(deltai, deltaj)#Pendiente respecto a la primera pieza

        if abs(deltai) >= self.X or abs(deltaj) >= self.X:return False #No basta con estar alineados, no puede haber huecos en la semirrecta
        if abs(currentSlope) != 1 and currentSlope != 0 and not np.isinf(currentSlope): return False #La pendiente de la semirrecta solo puede tomar los valores 0, +-1 y Inf

        for i in range(2,self.X):
            secondi, secondj = posiciones[i]
            deltaj, deltai = (secondj - firstj), (secondi - firsti)
            slope = np.divide(deltai, deltaj)#Pendiente respecto a la primera pieza              
            if slope != currentSlope:return False #La pendiente de toda la recta debe ser la misma
            if abs(deltai) >= self.X or abs(deltaj) >= self.X:return False #No basta con estar alineados, no puede haber huecos en la semirrecta

        # Es una buena práctica restablecer el warning de dividir entre cero
        np.seterr(divide='warn', invalid='warn')

        #Si nos hemos encontrado problemas tenemos nuestra semirrecta
        return True     
           
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError
    
    def mostrar_solucion(problem, node):
        """Imprime los estados paso a paso junto con las acciones."""
        #path = node.path()  
        estados = [problem.initial] 
        print("Pasos desde el estado inicial hasta el objetivo:\n")
        
        for action in node.solution(): 
            nuevo_estado = problem.result(estados[-1], action)  
            print(f"Acción: {action} - Estado: {nuevo_estado}")
            estados.append(nuevo_estado)  
    print("\n")
    
    def InicializarEstadoInicial(self,tablero, data):
        cont = 0
        N = len(tablero)
        for i in range(N):
            for j in range(N):
                var = tablero[i][j]
                if var != "_":
                    data[cont][0]= var
                    data[cont][1]= (i, j)
                    cont += 1
    

    # Método auxiliar para ver si una celda está vacía
    def is_empty(self, state, position):
        piezas_posiciones = state.state['location']
        exists = np.any(np.all(piezas_posiciones == position, axis=1))
        return not exists
    

    

    

In [3]:
def alineacionCentral_manhattan(node):
        #tamaño del tablero (NxN)
        N = len(node.state)
        print("N es: ", len(node.state))
        #creo un punto de alineación de referencia en el centro del tablero, y para calcularlo divido entre 2 las coordenadas X e Y
        alineada_x, alienada_y = N //2, N //2 #uso la doble barra (//) para que no pille los decimales

        print("entrooo\n")
        
        #distancia total al la linea de meta, en este caso el centro del tablero
        distancia_total = 0

        print("sige\n")

        #se rrecorren todas las posiciones de las fichas en el estado actual
        #print("node.state.state es:  ", node.state.state)
        print("node.state es:  ", node.state)
        
        for pieza, posicion in node.state: #hago el for con elemento (que es la tupla de tipo de ficha y posicion) porque si meto una tupla directamente peta el kernel
            #calclulo la distancia desde su posicion incial hasta el punto de alineación en el centro del tablero
            x, y = posicion
            distancia = abs(x - alineada_x) + abs(y - alienada_y)
            print(distancia)
            distancia_total += distancia #sumo la distancia a la distancia total"""

        return distancia_total #devuelvo la distancia total como valor heurístico



In [4]:
from search import astar_search
# estado inicial

tableroDiagonal = [
    ['S', '_', '_', '_', '_'],
    ['_', 'V', '_', '_', '_'],
    ['_', '_', '_', '_', '_'],
    ['_', '_', 'L', 'H', '_'],
    ['_', '_', '_', '_', '_']
]

N = 4 #numero d piezas

# Crear instancia del problema MovIX
hola2 = MovIX(tableroDiagonal, numpieces = N)


#astar_search(hola2).solution()


# Búsqueda A* con la heurística alineacionCentral_manhattan
solution2 = astar_search(hola2, h=alineacionCentral_manhattan, display=True)


# Mostrar solución si se encuentra
if solution2:
    print("Solución encontrada:")
    print(solution2)
else:
    print("No se encontró solución.")

#"""

N es:  4
entrooo

sige

node.state es:   <Node [('S', [0, 0]) ('V', [1, 1]) ('L', [3, 2]) ('H', [3, 3])]>


TypeError: 'MovIXNode' object is not iterable

In [4]:
from search import astar_search
# estado inicial

tableroDiagonal = [
    ['S', '_', '_', '_', '_'],
    ['_', 'V', '_', '_', '_'],
    ['_', '_', '_', '_', '_'],
    ['_', '_', 'L', 'H', '_'],
    ['_', '_', '_', '_', '_']
]

N = 4 #numero d piezas


hola = MovIX(tableroDiagonal, numpieces=N)

# búsqueda DFS
%timeit sum(range(1000))
 #contar nodos
solution = breadth_first_tree_search(hola)

# mostrar solución
if solution:
    print("Solución encontrada:", solution.state)
else:
    print("No se encontró solución.")


6.97 μs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Solución encontrada: <Node [('S', [0, 0]) ('V', [1, 1]) ('L', [2, 2]) ('H', [3, 3])]>
